In [1]:
!pip install pandas numpy faker dash plotly scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 40.4 MB/s eta 0:00:00


In [2]:
! pip install LinearRegression

In [3]:
import pandas as pd
import numpy as np
from faker import Faker
import random
from sklearn.linear_model import LinearRegression
from datetime import timedelta

fake = Faker()
np.random.seed(24)

products = ['Phone', 'Dress', 'Headphones', 'Smartwatch' , 'Shoes']
categories = ['Electronics', 'Clothing']
regions = ['Riyadh' , 'Jeddah', 'Dammam', 'Mecca']
channels = ['Website', 'App', 'Store']

data = {
    'Transaction ID': ['T' + str(i).zfill(3) for i in range(1, 301)],
    'Date': pd.date_range(start='2025-01-01', end='2025-12-31', periods=300),
    'Product': [random.choice(products) for _ in range(300)],
    'Category': [random.choice(categories) for _ in range(300)],
    'Quantity': np.random.randint(1, 20, 300),
    'Unit Price': np.random.randint(50, 5000, 300),
    'Region': [random.choice(regions) for _ in range(300)]
}

df = pd.DataFrame(data)
df['Total Sales'] = df['Quantity'] * df['Unit Price']
df['Month'] = df['Date'].dt.strftime('%B')

from dash import Dash, dcc, html, Input, Output
import plotly.express as px

app = Dash(__name__)

app.layout = html.Div([
    html.H1('لوحة تحليل بيانات المبيعات مع الذكاء الاصطناعي', style={'textAlign': 'center', 'color': '#ffffff'}),
    dcc.Dropdown(
        id='region-filter',
        options=[{'label': region, 'value': region} for region in df['Region'].unique()],
        value=None,
        placeholder='اختر المنطقة',
        style={'width': '50%', 'margin': 'auto'}

    ),
    dcc.Graph(id='line-chart'),
    dcc.Graph(id='bar-chart'),
    dcc.Graph(id='pie-chart'),
    dcc.Graph(id='prediction-chart')
], style={'backgroundColor': '#1a1a1a', 'padding': '20px'})

@app.callback(
    [Output('line-chart', 'figure'),
     Output('bar-chart', 'figure'),
     Output('pie-chart', 'figure'),
     Output('prediction-chart', 'figure')],
    [Input('region-filter', 'value')]
)

def update_charts(region):
    filtered_df = df if region is None else df[df['Region'] == region]

    monthly_sales = filtered_df.groupby('Month')['Total Sales'].sum().reset_index()

    month_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
    monthly_sales['Month'] = pd.Categorical(monthly_sales['Month'], categories=month_order, ordered=True)
    monthly_sales = monthly_sales.sort_values('Month')

    fig_line = px.line(monthly_sales, x='Month', y='Total Sales', title='اتجاه المبيعات الشهرية',
                         labels={'Total Sales': ' اجمالي المبيعات(ريال)', 'Month': 'الشهر'})
    fig_line.update_layout(template='plotly_dark')

    product_sales = filtered_df.groupby('Product')['Total Sales'].sum().reset_index()
    fig_bar = px.bar(product_sales, x='Product', y='Total Sales', title='اجمالي المبيعات لكل منتج',
                     labels={'Total Sales': ' اجمالي المبيعات(ريال)', 'Product': 'المنتج'})
    fig_bar.update_layout(template='plotly_dark')

    category_sales = filtered_df.groupby('Category')['Total Sales'].sum().reset_index() # Corrected variable name
    fig_pie = px.pie(category_sales, values='Total Sales', names='Category', title='نوزيع البيانات حسب الفئة')
    fig_pie.update_layout(template='plotly_dark')

    months_numeric = pd.to_datetime(monthly_sales['Month'], format='%B').map(lambda x: x.month)
    X = months_numeric.values.reshape(-1, 1)
    y = monthly_sales['Total Sales'].values
    model = LinearRegression()
    model.fit(X, y)


    future_months_numeric = np.array(range(13, 25)).reshape(-1, 1)
    predicted_sales = model.predict(future_months_numeric)


    future_month_dates = pd.date_range(start='2026-01-01', periods=12, freq='M')
    future_month_names = future_month_dates.strftime('%B')


    future_df = pd.DataFrame({
        'Month': future_month_names,
        'Total Sales': predicted_sales
    })

    combined_df = pd.concat([monthly_sales, future_df])


    fig_prediction = px.line(combined_df, x='Month', y='Total Sales', title='(2026-2025)تنبؤ المبيعات الشهرية',
                             labels={'Total Sales': ' اجمالي المبيعات(ريال)', 'Month': 'الشهر'})
    fig_prediction.add_scatter(x=future_df['Month'], y=future_df['Total Sales'], mode='lines', name='Predicted',
                              line=dict(color='red', dash='dash'))
    fig_prediction.update_layout(template='plotly_dark')


    return fig_line, fig_bar, fig_pie, fig_prediction

from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(8050)"))

app.run(debug=True, host='0.0.0.0', port=8050)

https://8050-m-s-3gl69r0ktn2gp-a.us-east4-0.prod.colab.dev


<IPython.core.display.Javascript object>